In [1]:
%load_ext autoreload
%load_ext line_profiler
%matplotlib qt
%autoreload 2

In [2]:
import os
os.chdir("..")
print(os.getcwd())

C:\Users\eac84\Universiteit Antwerpen\Thermography - Biomedical thermography\src


In [4]:
from retna.segment import segment_data
from thermography_python.base import ThermoData
from thermography_python.render import render_video
import glob

In [16]:
from thermography_python.io import read_H5

In [5]:

import torch

In [6]:
root = "T:\ProjectData\DEIP FLAP"

In [20]:
H5_fns = glob.glob(root + "\\*\\*.H5")
H5_fns

['T:\\ProjectData\\DEIP FLAP\\07_01_2020\\anastomose, ingehecht.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\anastomose.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\intra1 re beide toe en li open.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\intra1 re beide toe geklipt en li open extra cooling.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\intra1 re beide toe geklipt en li open.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\pre op 1.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\pre op met cooling.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_01_2020\\pre op met perfos.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_02_2020\\anastomose linker borst.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_02_2020\\anastomose rechter borst.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_02_2020\\late meting op buik re.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_02_2020\\li flap na afklemmen.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_02_2020\\per op met klemmen re flap.H5',
 'T:\\ProjectData\\DEIP FLAP\\07_02_2020\\pre op met cooling laat.H

In [25]:
model_fn = "./models/Flap+Pref3.pt"
model = torch.load(model_fn)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [29]:

for fn in H5_fns:
    
    print(fn)
    IRT_im = ThermoData(fn)
    frames = segment_data(model, IRT_im, nFrames=10)
    print("")   
    fn_out = ("_").join(fn.split("\\")[-2:]).split(".")[0] + "_colored.mp4" 
    fn_out = "..//output//" + fn_out
    
    if os.path.exists(fn_out):    os.remove(fn_out)
    render_video( fn_out, frames )
        

T:\ProjectData\DEIP FLAP\07_01_2020\anastomose, ingehecht.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\anastomose.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\intra1 re beide toe en li open.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\intra1 re beide toe geklipt en li open extra cooling.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\intra1 re beide toe geklipt en li open.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\pre op 1.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\pre op met cooling.H5
..........
T:\ProjectData\DEIP FLAP\07_01_2020\pre op met perfos.H5
..........
T:\ProjectData\DEIP FLAP\07_02_2020\anastomose linker borst.H5
..........
T:\ProjectData\DEIP FLAP\07_02_2020\anastomose rechter borst.H5
..........
T:\ProjectData\DEIP FLAP\07_02_2020\late meting op buik re.H5
..........
T:\ProjectData\DEIP FLAP\07_02_2020\li flap na afklemmen.H5
..........
T:\ProjectData\DEIP FLAP\07_02_2020\per op met klemmen re flap.H5
..........
T:\ProjectData\DEIP FLAP\07_

In [18]:
%lprun -f read_H5 segment_data(model, IRT_im, nFrames=3)

...

## SANDBOX

In [165]:
def norm_uint8(d):
    d = d - np.min(d)
    d = d / np.max(d) 
    d = (d*255).astype(np.uint8)
    return d

def seg_flap(data):
    
    d = data[330]
    d = norm_uint8(d)
    bmsk = d<25
    bmsk = mrp.remove_small_objects(bmsk,100)    
    bmsk = mrp.binary_closing(bmsk, mrp.disk(50))
    bmsk = mrp.binary_erosion(bmsk , mrp.disk(50))
    #bmsk  = mrp.binary_opening(bmsk , mrp.disk(20))
    
    return bmsk

def seg_vessels(d,b_msk):
    
    im = norm_uint8(d)
    bseg = im > np.mean(im[b_msk])
    b_ves = b_msk & ~bseg
    b_ves  = mrp.binary_erosion(b_ves , mrp.disk(3))
    b_ves = mrp.remove_small_objects(b_ves,100)    
    
    return b_ves

def draw_vessels(fn):
    
    data = read_thermo(fn)
    bmsk = seg_flap(data)

    frames = []
    for idx in np.linspace(250,len(data)-1,100):

        d = data[int(idx)]
        bV = seg_vessels(d,bmsk)

        d = norm_uint8(d)
        imin_3 = d[...,None]*[1,1,1]
        imin_3 = imin_3.astype(np.uint8)

        label = ndi.label(bV)[0]
        shapes = seg.label_to_shapes(label, kind="poly", tol=1)
        labl_3 = seg.draw_on_im( imin_3, shapes)

        frames.append(labl_3)

    path_out = "..//output_ves//"
    fn_out = ("_").join(fn.split("\\")[-2:]).split(".")[0] + "_colored.mp4" 
    fn_out = path_out + fn_out

    if not os.path.exists(path_out):    os.mkdir(path_out)
    if os.path.exists(fn_out):    os.remove(fn_out)
    render_frames( fn_out, frames )    

In [6]:
## Selecting frames
root = "C:\\Users\\eac84\\Desktop\\FLAP\\*\\"
fns = glob.glob("./training/*input.tiff")
fns = [fn.split("\\")[-1].split("__")[1].split("_f")[0] for fn in fns]

xvi_fns = [glob.glob(root+fn+".xvi") for fn in fns]
xvi_fns = list({fn for fns in xvi_fns for fn in fns})

IndexError: list index out of range